In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, InputLayer, Input, Flatten, Conv1D, MaxPooling1D, BatchNormalization
import optuna
import openpyxl
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from keras.losses import SparseCategoricalCrossentropy
from keras.losses import MeanAbsoluteError,mean_squared_error

In [ ]:
ds = pd.read_csv('intraindata2n.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_train = np.array(ds.iloc[:, 5:])
y1_train = np.array(ds[[0]])
y2_train = np.array(ds[[1]])
y3_train = np.array(ds[[2]])
y4_train = np.array(ds[[3]])
y5_train = np.array(ds[[4]])
ds = pd.read_csv('invalidation2n.csv', header=None)
ds = ds.sample(frac=1).reset_index(drop=True)
x_val = np.array(ds.iloc[:, 5:])
y1_val = np.array(ds[[0]])
y2_val = np.array(ds[[1]])
y3_val = np.array(ds[[2]])
y4_val = np.array(ds[[3]])
y5_val = np.array(ds[[4]])

In [ ]:
miny3 = np.min(y3_train)
maxy3 = np.max(y3_train)
miny4 = np.min(y4_train)
maxy4 = np.max(y4_train)
miny5 = np.min(y5_train)
maxy5 = np.max(y5_train)
miny31 = np.min(y3_val)
maxy31= np.max(y3_val)
miny41 = np.min(y4_val)
maxy41 = np.max(y4_val)
miny51 = np.min(y5_val)
maxy51 = np.max(y5_val)
y3_train = (y3_train - miny3) / (maxy3 - miny3)
y4_train = (y4_train - miny4) / (maxy4 - miny4)
y5_train = (y5_train - miny5) / (maxy5 - miny5)



y3_val = (y3_val- miny31) / (maxy31 - miny31)
y4_val = (y4_val - miny41) / (maxy41- miny41)
y5_val = (y5_val - miny51) / (maxy51 - miny51)

In [ ]:
forward_model = keras.models.load_model('forward3.h5')
forward_model.trainable = False

In [ ]:
x1=Input(shape=x_train[0].shape)


In [ ]:
in1 = Input(shape=x_train[0].shape)

x1 = Dense(500, activation='relu')(in1)
x1 = Dense(500, activation='relu')(x1)
x1 = Dense(500, activation='relu')(x1)
x1 = Dense(500, activation='relu')(x1)
out1 = Dense(2, activation='softmax')(x1)

x2 = Dense(800, activation='relu')(in1)
x2 = Dense(800, activation='relu')(x2)
x2 = Dense(800, activation='relu')(x2)
x2 = Dense(800, activation='relu')(x2)
out2 = Dense(3, activation='softmax')(x2)

xa = Dense(1180, activation='relu')(in1)
xa = Dense(1180, activation='relu')(xa)
xa = Dense(1180, activation='relu')(xa)
xa = Dense(1180, activation='relu')(xa)
xa = Dense(1180, activation='relu')(xa)
xa = Dense(1180, activation='relu')(xa)
out3 = Dense(1, activation='linear')(xa)
xb = Dense(1180, activation='relu')(in1)
xb = Dense(1180, activation='relu')(xb)
xb = Dense(1180, activation='relu')(xb)
xb = Dense(1180, activation='relu')(xb)
xb = Dense(1180, activation='relu')(xb)
xb = Dense(1180, activation='relu')(xb)
out4 = Dense(1, activation='linear')(xb)

xc = Dense(1180, activation='relu')(in1)
xc = Dense(1180, activation='relu')(xc)
xc = Dense(1180, activation='relu')(xc)
xc = Dense(1180, activation='relu')(xc)
xc = Dense(1180, activation='relu')(xc)
xc = Dense(1180, activation='relu')(xc)
out5 = Dense(1, activation='linear')(xc)
inverse_model = Model(inputs=in1, outputs=(out1, out2, out3, out4, out5))


In [ ]:
def merge(out1, out2, out3, out4, out5):
    argmax=tf.math.argmax(out2, axis=1)
    arg=tf.expand_dims(argmax, axis=1)

    argmax1=tf.math.argmax(out1, axis=1)
    arg1=tf.expand_dims(argmax1, axis=1)

    outf1 = tf.cast(arg1, dtype=tf.float32)
    outf2 = tf.cast(arg, dtype=tf.float32)
    merge=tf.concat([outf1, outf2, out3, out4, out5], axis=1)
    return merge


In [ ]:
in1 = Input(shape=x_train[0].shape)
inverse_out=inverse_model(in1)
ou1 = inverse_out[0]
ou2 = inverse_out[1]
ou3 = inverse_out[2]
ou4 = inverse_out[3]
ou5 = inverse_out[4]
outf=merge(ou1, ou2, ou3, ou4, ou5)
print(outf.shape)
interout=forward_model(outf)
tandem_out=interout
tandem = keras.models.Model(inputs=in1, outputs=(tandem_out, ou1, ou2, ou3, ou4, ou5))

print("y1_train dtype:", ou1.dtype)
print("y2_train dtype:", ou2.dtype)

(None, 5)
y1_train dtype: <dtype: 'float32'>
y2_train dtype: <dtype: 'float32'>


In [ ]:
optimizer=tf.keras.optimizers.Adam(0.000325)
tandem.compile(optimizer=optimizer, loss=['mse','SparseCategoricalCrossentropy', 'SparseCategoricalCrossentropy', 'mse', 'mse', 'mse'],
              metrics=[['mse'], ['accuracy'], ['accuracy'], ['mse','mae'], ['mse','mae'], ['mse','mae']])
early=EarlyStopping(monitor='val_loss', patience=100, mode='min', restore_best_weights=True)


In [ ]:
history = tandem.fit(x_train, (x_train, y1_train, y2_train, y3_train, y4_train, y5_train),validation_data=(x_val,[x_val, y1_val, y2_val, y3_val, y4_val, y5_val]),

                     batch_size=128, epochs=1000, validation_batch_size=128, verbose=2, shuffle=True, callbacks=[ early]
                     )

In [ ]:
val_loss = history.history['loss']
df1=pd.DataFrame(val_loss)
df1.to_excel('tanloss3.xlsx')
loss=history.history['sequential_loss']
df2=pd.DataFrame(loss)
df2.to_excel('tanseqloss3.xlsx')
val_loss = history.history['val_loss']
df3=pd.DataFrame(val_loss)
df3.to_excel('tanvalidation_loss3.xlsx')
loss=history.history['val_sequential_loss']
df4=pd.DataFrame(loss)
df4.to_excel('tanvalseqloss3.xlsx')


In [ ]:
tandem.save('models/inversenet_tandem_full3.h5')
inverse_model.save('inversenet_tandem_generator3.h5')